In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()
OpenAI_API_Key = os.getenv("OPENAI_API_KEY")
# print(OpenAI_API_Key)

In [146]:
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import json
import pandas as pd
import traceback
import PyPDF2

In [147]:
llm = ChatOpenAI(model = 'gpt-3.5-turbo')

In [148]:
with open("i:\8_end_to_end_projects\Project_2\MCQ_Generator\Response.json", 'r') as f:
    Response_JSON = json.load(f)

print(Response_JSON)

{'Question 1': {'mcq': 'Multiple Choice Question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'Correct': 'correct answer'}, 'Question 2': {'mcq': 'Multiple Choice Question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'Correct': 'correct answer'}, 'Question 3': {'mcq': 'Multiple Choice Question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'Correct': 'correct answer'}, 'Question 4': {'mcq': 'Multiple Choice Question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'Correct': 'correct answer'}, 'Question 5': {'mcq': 'Multiple Choice Question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'Correct': 'correct answer'}}


In [149]:
main_template="""
Text:{topic}
You are an expert MCQ generator. Given the above Topic, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the topic as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs

### RESPONSE_JSON
{response_json}

"""

In [150]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["topic", "number", "subject", "tone", "response_json"],
    template=main_template
    )

In [151]:
quiz_generation_chain = LLMChain(llm=llm, prompt = quiz_generation_prompt, verbose = True, output_key= "quiz")

In [132]:
Quiz = quiz_generation_chain.run(topic = 'Supervised Machine Learning', number =5, subject = 'Machine learning', tone = 'academic', response_json = Response_JSON)



> Entering new LLMChain chain...
Prompt after formatting:

Text:Supervised Machine Learning
You are an expert MCQ generator. Given the above Topic, it is your job to create a quiz  of 5 multiple choice questions for Machine learning students in academic tone. 
Make sure the questions are not repeated and check all the questions to be conforming the topic as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make 5 MCQs

### RESPONSE_JSON
{'Question 1': {'mcq': 'Multiple Choice Question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'Correct': 'correct answer'}, 'Question 2': {'mcq': 'Multiple Choice Question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'Correct': 'correct answer'}, 'Question 3': {'mcq': 'Multiple Choice Question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'Correct': 'corre


> Finished chain.


In [133]:
print(Quiz)


{
  "Question 1": {
    "mcq": "What is Supervised Machine Learning?",
    "options": {
      "a": "A type of machine learning where the model learns from labeled data",
      "b": "A type of machine learning where the model learns from unlabeled data",
      "c": "A type of machine learning where the model doesn't require any data",
      "d": "A type of machine learning where the model only uses images"
    },
    "Correct": "a"
  },
  "Question 2": {
    "mcq": "Which of the following is an example of Supervised Machine Learning algorithm?",
    "options": {
      "a": "K-means clustering",
      "b": "Decision Tree",
      "c": "Principal Component Analysis",
      "d": "Apriori Algorithm"
    },
    "Correct": "b"
  },
  "Question 3": {
    "mcq": "In Supervised Machine Learning, what is the goal of the model?",
    "options": {
      "a": "To make predictions on unlabeled data",
      "b": "To learn patterns from labeled data",
      "c": "To group similar data points together",

In [134]:
verification_template="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the {tone} such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [135]:
quiz_verification_prompt = PromptTemplate(
    input_variables=["subject", "quiz", "tone"],
    template=verification_template
    )

In [136]:
quiz_verification_chain = LLMChain(llm=llm, prompt = quiz_verification_prompt, verbose = True, output_key= "review")

In [137]:
Review = quiz_verification_chain.run(subject = 'Machine learning',  quiz = Quiz, tone = 'academic')



> Entering new LLMChain chain...
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for Machine learning students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the academic such that it perfectly fits the student abilities
Quiz_MCQs:

{
  "Question 1": {
    "mcq": "What is Supervised Machine Learning?",
    "options": {
      "a": "A type of machine learning where the model learns from labeled data",
      "b": "A type of machine learning where the model learns from unlabeled data",
      "c": "A type of machine learning where the model doesn't require any data",
      "d": "A type of machine learning where the model only uses images"
    },
    "Correct": "a"
  },
  "Question 2": {
    


> Finished chain.


In [138]:
Review

'The complexity of the quiz is suitable for machine learning students as it covers basic concepts of supervised machine learning. The questions are clear and concise, testing understanding of key terms and concepts. No changes are needed.'

In [139]:
overall_chain = SequentialChain(chains = [quiz_generation_chain, quiz_verification_chain], input_variables=["topic", "number", "subject", "tone", "response_json"], output_variables= ['quiz', 'review'])

In [140]:
overall_chain.output_keys

['quiz', 'review']

In [154]:
Response_JSON

{'Question 1': {'mcq': 'Multiple Choice Question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'Correct': 'correct answer'},
 'Question 2': {'mcq': 'Multiple Choice Question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'Correct': 'correct answer'},
 'Question 3': {'mcq': 'Multiple Choice Question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'Correct': 'correct answer'},
 'Question 4': {'mcq': 'Multiple Choice Question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'Correct': 'correct answer'},
 'Question 5': {'mcq': 'Multiple Choice Question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'Correct': 'correct answer'}}

In [152]:
json.dumps(Response_JSON)

'{"Question 1": {"mcq": "Multiple Choice Question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "Correct": "correct answer"}, "Question 2": {"mcq": "Multiple Choice Question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "Correct": "correct answer"}, "Question 3": {"mcq": "Multiple Choice Question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "Correct": "correct answer"}, "Question 4": {"mcq": "Multiple Choice Question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "Correct": "correct answer"}, "Question 5": {"mcq": "Multiple Choice Question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "Correct": "correct answer"}}'

In [141]:
with get_openai_callback() as cb:
    Final_output=overall_chain(
        {'topic':'Supervised Machine Learning', 
        'number': 5, 
        'subject' : 'Machine learning', 
        'tone' :'academic',
        'response_json' : json.dumps(Response_JSON)}
        )



> Entering new LLMChain chain...
Prompt after formatting:

Text:Supervised Machine Learning
You are an expert MCQ generator. Given the above Topic, it is your job to create a quiz  of 5 multiple choice questions for Machine learning students in academic tone. 
Make sure the questions are not repeated and check all the questions to be conforming the topic as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make 5 MCQs

### RESPONSE_JSON
{"Question 1": {"mcq": "Multiple Choice Question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "Correct": "correct answer"}, "Question 2": {"mcq": "Multiple Choice Question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "Correct": "correct answer"}, "Question 3": {"mcq": "Multiple Choice Question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "Correct": "corre


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for Machine learning students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the academic such that it perfectly fits the student abilities
Quiz_MCQs:

{
  "Question 1": {
    "mcq": "What is Supervised Machine Learning?",
    "options": {
      "a": "A type of machine learning where the model learns from labeled data",
      "b": "A type of machine learning where the model learns from unlabeled data",
      "c": "A type of machine learning where the model does not require any data",
      "d": "A type of machine learning where the model learns from reinforcement"
    },
    "Correct": "a

In [153]:
Final_output

{'topic': 'Supervised Machine Learning',
 'number': 5,
 'subject': 'Machine learning',
 'tone': 'academic',
 'response_json': '{"Question 1": {"mcq": "Multiple Choice Question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "Correct": "correct answer"}, "Question 2": {"mcq": "Multiple Choice Question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "Correct": "correct answer"}, "Question 3": {"mcq": "Multiple Choice Question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "Correct": "correct answer"}, "Question 4": {"mcq": "Multiple Choice Question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "Correct": "correct answer"}, "Question 5": {"mcq": "Multiple Choice Question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "Correct": "correct answer"}}',
 'qui

In [155]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1464
Prompt Tokens:949
Completion Tokens:515
Total Cost:0.0024535


In [161]:
quiz=Final_output.get("quiz")

In [162]:
quiz_dict=json.loads(quiz)

In [163]:
quiz_dict

{'Question 1': {'mcq': 'What is Supervised Machine Learning?',
  'options': {'a': 'A type of machine learning where the model learns from labeled data',
   'b': 'A type of machine learning where the model learns from unlabeled data',
   'c': 'A type of machine learning where the model does not require any data',
   'd': 'A type of machine learning where the model learns from reinforcement'},
  'Correct': 'a'},
 'Question 2': {'mcq': 'What is the goal of Supervised Machine Learning?',
  'options': {'a': 'To predict future outcomes based on historical data',
   'b': 'To classify data into different categories',
   'c': 'To optimize a function',
   'd': 'To learn patterns from unstructured data'},
  'Correct': 'a'},
 'Question 3': {'mcq': 'Which of the following is an example of Supervised Machine Learning algorithm?',
  'options': {'a': 'K-means clustering',
   'b': 'Random Forest',
   'c': 'Principal Component Analysis',
   'd': 'Apriori algorithm'},
  'Correct': 'b'},
 'Question 4': {'

In [178]:
quiz_table_data = []
for key, value in quiz_dict.items():
    mcq = value['mcq']
    option_a = value['options']['a']
    option_b = value['options']['b']
    option_c = value['options']['c']
    option_d = value['options']['d']
    correct = value['Correct']
    quiz_table_data.append({'Question': mcq, "Options A": option_a, "Options B": option_b, "Options C": option_c, "Options D": option_d, "Correct Answer": correct})

In [179]:
quiz_table_df = pd.DataFrame(quiz_table_data)

In [180]:
display(quiz_table_df)

,Question,Options A,Options B,Options C,Options D,Correct Answer
0,What is Supervised Machine Learning?,A type of machine learning where the model lea...,A type of machine learning where the model lea...,A type of machine learning where the model doe...,A type of machine learning where the model lea...,a
1,What is the goal of Supervised Machine Learning?,To predict future outcomes based on historical...,To classify data into different categories,To optimize a function,To learn patterns from unstructured data,a
2,Which of the following is an example of Superv...,K-means clustering,Random Forest,Principal Component Analysis,Apriori algorithm,b
3,What is the process of Supervised Machine Lear...,"Feature selection, data preprocessing, model t...","Model evaluation, feature selection, data prep...","Data preprocessing, model evaluation, feature ...","Model training, model evaluation, feature sele...",a
4,Which evaluation metric is commonly used in Su...,Mean Squared Error (MSE),Accuracy,R-squared,Mean Absolute Error (MAE),b


In [177]:
quiz_table_df.to_csv('Supervised_Machine_Learning_quiz.csv', index= False)